---
sidebar_position: 3
---

> You can get this [Notebook](https://github.com/Seeed-Projects/Tutorial-of-AI-Kit-with-Raspberry-Pi-From-Zero-to-Hero/blob/main/articles/Chapter_5-Custom_Model_Development_and_Deployment/Deploy_Your_Model.ipynb) on GitHub.

# Step 1: Prepare you environment on your raspberry pi

**Note：This part of code run on your [raspberry pi5](https://www.seeedstudio.com/Raspberry-Pi-5-2GB-p-5938.html) or [Recomputer R Series](https://www.seeedstudio.com/reComputer-R1000-Series-Optional-Accessories.html) with your [AI kit](https://www.seeedstudio.com/Raspberry-Pi-AI-Kit-p-5900.html)**

## Update time

```
sudo date -s "$(wget -qSO- --max-redirect=0 google.com 2>&1 | grep Date: | cut -d' ' -f5-8)Z"
```


## Upgrade raspberry pi

```
sudo apt update && sudo apt upgrade
```

## Install jupyter 

```
sudo apt install jupyter
```

## Open jupyter

```
jupyter notebook
```

>**Note**
Please install hailo8l to your raspberrypi follow [this](https://www.raspberrypi.com/documentation/accessories/ai-kit.html), and for [reComputerR](https://www.seeedstudio.com/reComputer-R1000-Series-Optional-Accessories.html) series follow [this](https://wiki.seeedstudio.com/yolov8_object_detection_on_recomputer_r1000_with_hailo_8l/#step-1-install-ai-kit).

In [1]:
# Install hailo-all on raspberrypi

! sudo apt install hailo-all -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
hailo-all is already the newest version (4.18.0+1).
You might want to run 'apt --fix-broken install' to correct these.
The following packages have unmet dependencies:
 hailo-all : Depends: hailofw (>= 4.18.0) but it is not going to be installed
E: Unmet dependencies. Try 'apt --fix-broken install' with no packages (or specify a solution).


In [2]:
# Check if the hailo-all is installed

! hailortcli fw-control identify

Executing on device: 0000:01:00.0
Identifying board
Control Protocol Version: 2
Firmware Version: 4.18.0 (release,app,extended context switch buffer)
Logger Version: 0
Board Name: Hailo-8                         
Device Architecture: HAILO8L
Serial Number: HLDDLBB243200427
Part Number: HM21LB1C2LAE
Product Name: HAILO-8L AI ACC M.2 B+M KEY MODULE EXT TMP



In [3]:
# Install hailo-rpi5-examples

! git clone https://github.com/hailo-ai/hailo-rpi5-examples.git

Cloning into 'hailo-rpi5-examples'...
remote: Enumerating objects: 487, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 487 (delta 117), reused 91 (delta 86), pack-reused 307 (from 1)
Receiving objects: 100% (487/487), 20.36 MiB | 9.76 MiB/s, done.
Resolving deltas: 100% (274/274), done.


# Step 2: Configure labels

The output of yolov8n hef model will be three classes 0, 1, 2. And we use `json` file to parse the output.

In [4]:
import json

data = {
    "detection_threshold": 0.1,
    "max_boxes": 200,
    "labels": [
        " ",
        "apple",
        "banana",
        "orange"
    ]
}

output_file = "config.json"

with open(output_file, "w") as file:
    json.dump(data, file, indent=4)

print(f"JSON file saved as {output_file}")

JSON file saved as config.json


In [5]:
# Install necessary libs

! source ./hailo-rpi5-examples/setup_env.sh && pip install setproctitle

Setting up the environment...
Setting up the environment for hailo-tappas-core...
TAPPAS_VERSION is 3.29.1. Proceeding...
You are not in the venv_hailo_rpi5_examples virtual environment.
Virtual environment does not exist. Creating and activating...
TAPPAS_POST_PROC_DIR set to /usr/lib/aarch64-linux-gnu/hailo/tappas/post_processes
DEVICE_ARCHITECTURE is set to: HAILO8L
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Using cached setproctitle-1.3.4-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (31 kB)


In [6]:
# Install postprocess .so lib

! source ./hailo-rpi5-examples/setup_env.sh && cd ./hailo-rpi5-examples && bash ./install.sh

Setting up the environment...
Setting up the environment for hailo-tappas-core...
TAPPAS_VERSION is 3.29.1. Proceeding...
You are not in the venv_hailo_rpi5_examples virtual environment.
Virtual environment exists. Activating...
TAPPAS_POST_PROC_DIR set to /usr/lib/aarch64-linux-gnu/hailo/tappas/post_processes
DEVICE_ARCHITECTURE is set to: HAILO8L
Setting up the environment...
Setting up the environment for hailo-tappas-core...
TAPPAS_VERSION is 3.29.1. Proceeding...
You are in the venv_hailo_rpi5_examples virtual environment.
TAPPAS_POST_PROC_DIR set to /usr/lib/aarch64-linux-gnu/hailo/tappas/post_processes
DEVICE_ARCHITECTURE is set to: HAILO8L
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (41.7 MB)
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Using cached https://www.piwheels.org/simple/pytest/pytest-8.3.3-py3-none-any.wh

C compiler for the host machine: cc (gcc 12.2.0 "cc (Debian 12.2.0-14) 12.2.0")
C linker for the host machine: cc ld.bfd 2.40
C++ compiler for the host machine: c++ (gcc 12.2.0 "c++ (Debian 12.2.0-14) 12.2.0")
C++ linker for the host machine: c++ ld.bfd 2.40
Host machine cpu family: aarch64
Host machine cpu: aarch64
Found pkg-config: /usr/bin/pkg-config (1.8.1)
Run-time dependency hailo-tappas-core found: YES 3.29.1
Build targets in project: 4

hailo_rpi5_examples 1.1.1

  User defined options
    buildtype: release

Found ninja-1.11.1 at /usr/bin/ninja
[8/8] Linking target cpp/libyolov5seg_postprocess.so.so.p/yolov5seg.cpp.o                  
[0/1] Installing files.
Installing cpp/libyolo_hailortpp_postprocess.so to /home/seeed/Tutorial-of-AI-Kit-with-Raspberry-Pi-From-Zero-to-Hero/articles/Chapter_5-Custom_Model_Development_and_Deployment/hailo-rpi5-examples/resources
Installing cpp/libremove_labels.so to /home/seeed/Tutorial-of-AI-Kit-with-Raspberry-Pi-From-Zero-to-Hero/articles/Cha

# Step 3: Run your model


📌 Note: If you meet error like `max_desc_page_size given 16384 is bigger than hw max desc page size 4096` please check this [link](https://community.hailo.ai/t/hailort-error-check-failed-max-desc-page-size-given-16384-is-bigger-than-hw-max-desc-page-size-4096/3690)

In [7]:
! source ./hailo-rpi5-examples/setup_env.sh && python ./hailo-rpi5-examples/basic_pipelines/detection_pipeline.py --labels-json ./config.json --hef ../../models/Chapter5/yolov8n.hef -i /dev/video0

Setting up the environment...
Setting up the environment for hailo-tappas-core...
TAPPAS_VERSION is 3.29.1. Proceeding...
You are not in the venv_hailo_rpi5_examples virtual environment.
Virtual environment exists. Activating...
TAPPAS_POST_PROC_DIR set to /usr/lib/aarch64-linux-gnu/hailo/tappas/post_processes
DEVICE_ARCHITECTURE is set to: HAILO8L
Auto-detected Hailo architecture: hailo8l
v4l2src device=/dev/video0 name=source ! video/x-raw, width=640, height=480 !  queue name=source_scale_q leaky=no max-size-buffers=3 max-size-bytes=0 max-size-time=0  ! videoscale name=source_videoscale n-threads=2 ! queue name=source_convert_q leaky=no max-size-buffers=3 max-size-bytes=0 max-size-time=0  ! videoconvert n-threads=3 name=source_convert qos=false ! video/x-raw, format=RGB, pixel-aspect-ratio=1/1 !  queue name=inference_scale_q leaky=no max-size-buffers=3 max-size-bytes=0 max-size-time=0  ! videoscale name=inference_videoscale n-threads=2 qos=false ! queue name=inference_convert_q leaky